# Titanic

Le but est de prédire si un passager du Titanic a survécu ou pas selon ses caractéristiques (son sexe, sa classe passager...).

## Chargement des données

In [1]:
import numpy as np 
import pandas as pd 

train=pd.read_csv('../input/titanic/train.csv')
test=pd.read_csv('../input/titanic/test.csv')
submission=pd.read_csv('../input/titanic/gender_submission.csv')

test

## Visualisation des données

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.heatmap(train.isnull(),yticklabels=False,cbar=False)

On remarque qu'il y a énormément de données manquantes sur les cabines, on se débarassera donc de cette colonne plus tard car les données sont inexploitables. Il y a également des données manquantes concernant l'âge des passagers. On peut soit remplir ces données manquantes en calculant l'âge moyen des passagers, soit supprimer les lignes.

In [3]:
plt.figure(figsize=(10, 7))
sns.heatmap(train[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']].corr(), annot = True, fmt='.2f', cmap='coolwarm')

On remarque que la chance de survie d'un passager est corrélée à sa classe passager et au montant de son ticket.

In [4]:
plt.figure(figsize=(10, 7))
sns.countplot(x='Survived',data=train,palette='RdBu_r')

Il y a plus de morts que de survivants.

In [5]:
plt.figure(figsize=(10, 7))
sns.countplot(x='Survived',hue='Sex',data=train,palette='RdBu_r')

Les femmes ont plus de chances de survivre.

In [6]:
plt.figure(figsize=(10, 7))
sns.countplot(x='Survived',hue='Pclass',data=train,palette='rainbow')

Les passagers de troisième classe constituent la majorité des morts. Les premières classes sont majoritaires chez les survivants. Il y a a plus de survivants chez les troisièmes classes que les deuxièmes, ce qui est probablement dû au fait qu'ils soient plus nombreux.

In [7]:
plt.figure(figsize=(10, 7))
sns.boxplot(x='Pclass',y='Age',data=train,palette='winter')

Les premières classes sont les plus agées et les troisièmes les plus jeunes.

## Transformation des données

On modifie les données pour faciliter l'analyse. On transforme les données catégorielles (qualitatives) en données numériques (quantitatives) pour faciliter les calculs sur ces données. 

In [8]:
# On se débarasse des colonnes inutiles
train = train.drop(['PassengerId','Name','Ticket', 'Cabin'], axis=1)
test = test.drop(['Name','Ticket', 'Cabin'], axis=1)

# On convertit les données catégorielles

# Embarcation
train_embark = pd.get_dummies(train['Embarked'])
test_embark = pd.get_dummies(test['Embarked'])

train.drop(['Embarked'],axis=1,inplace=True)
test.drop(['Embarked'],axis=1,inplace=True)

train = pd.concat([train,train_embark],axis=1)
test = pd.concat([test, test_embark], axis=1)

# Sexe
sex_train = pd.get_dummies(train['Sex'],drop_first=True)
sex_test = pd.get_dummies(test['Sex'], drop_first=True)

train.drop(['Sex'],axis=1,inplace=True)
test.drop(['Sex'],axis=1,inplace=True)

train = pd.concat([train,sex_train],axis=1)
test = pd.concat([test, sex_test], axis=1)

# On supprime les lignes avec des valeurs manquantes
train = train.dropna()
test = test.dropna()


train.info()
print("-----------------------------------------")
test.info()

## Séparation des données en deux sous-ensembles pour les tests et l'apprentissage

In [9]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

train_X = train.drop('Survived', axis=1)
train_y = train['Survived']
test_X = test.drop('PassengerId', axis=1)

test = pd.merge(test, submission, on='PassengerId')
test_y = test.Survived

models_names = []
scores = []

## RandomForestRegressor

In [10]:
from sklearn.ensemble import RandomForestRegressor


model = RandomForestRegressor(random_state=1)
model.fit(train_X, train_y)
models_names.append("RandomForestRegressor")

test_predictions = model.predict(test_X)
score = model.score(train_X, train_y)
scores.append(score)
print("Score : " + str(score) + " | mae : " + str(mean_absolute_error(test_y, test_predictions)))


On obtient de moins bons résultats sur un jeu de données différent de celui utilisé pour l'entrainement, dû à l'overfitting.

## RandomForestClassifier

In [11]:
from sklearn.ensemble import RandomForestClassifier


model = RandomForestClassifier(random_state=1)
model.fit(train_X, train_y)
models_names.append("RandomForestClassifier")

test_predictions = model.predict(test_X)
score = model.score(train_X, train_y)
scores.append(score)
print("Score : " + str(score) + " | accuracy score : " + str(accuracy_score(test_y, test_predictions)))


## LinearRegression

In [12]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(train_X, train_y)
models_names.append("LinearRegression")

test_predictions = model.predict(test_X)
score = model.score(train_X, train_y)
scores.append(score)

## LogisticRegression

In [13]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=1, solver='liblinear')
model.fit(train_X, train_y)
models_names.append("LogisticRegression")

test_predictions = model.predict(test_X)
score = model.score(train_X, train_y)
scores.append(score)

## Support Vector Machines

In [14]:
from sklearn.svm import SVC

model = SVC()
model.fit(train_X, train_y)
models_names.append("SVC")

test_predictions = model.predict(test_X)
score = model.score(train_X, train_y)
scores.append(score)

## KNeighborsClassifier

In [15]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors = 2)
model.fit(train_X, train_y)
models_names.append("KNeighborsClassifier")

test_predictions = model.predict(test_X)
score = model.score(train_X, train_y)
scores.append(score)

## Gaussian Naive Bayes

In [16]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(train_X, train_y)
models_names.append("GaussianNB")

test_predictions = model.predict(test_X)
score = model.score(train_X, train_y)
scores.append(score)

## Gradient Boosting Classifier

In [17]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators=150)
model.fit(train_X, train_y)
models_names.append("GradientBoostingClassifier")

test_predictions = model.predict(test_X)
score = model.score(train_X, train_y)
scores.append(score)

## Comparaison des modèles

In [18]:
models = pd.DataFrame({'Model': models_names,
                       'Score': scores})

models.sort_values(by = 'Score', ascending = False, ignore_index=True)

Le modèle RandomForestClassifier a l'air d'être le meilleur modèle, mais comme vu plus haut, les performances des modèles varient selon les jeux de données utilisés. Les scores affichés en haut ont été obtenus en appliquant les modèles au jeu de données utilisé pour l'entrainement. Il y a donc un risque d'overfitting.

## Cross Validation

Pour palier au problème d'overfitting et avoir une meilleure idée des performandces des modèles, on utilise la cross validation.

In [19]:
from sklearn.model_selection import cross_val_score

models = [RandomForestRegressor(random_state=31),
          RandomForestClassifier(random_state=1),
          LinearRegression(),
          LogisticRegression(random_state=1, solver='liblinear'),
          SVC(),
          KNeighborsClassifier(n_neighbors = 2),
          GaussianNB(),
          GradientBoostingClassifier(n_estimators=150)]

scores = []
for m in models:
    scores.append(cross_val_score(m, train_X, train_y, cv=10))
    

cv_mean = []
cv_std = []
for s in scores:
    cv_mean.append(s.mean())
    cv_std.append(s.std())
    
cv_res = pd.DataFrame({'Cross Validation Mean': cv_mean, 'Cross Validation Std': cv_std, 'Model': models_names})
cv_res.sort_values(by = 'Cross Validation Mean', ascending = False, ignore_index = True)

In [20]:
sns.barplot(x = 'Cross Validation Mean', y = 'Model', data = cv_res, order = cv_res.sort_values(by = 'Cross Validation Mean', ascending = False)['Model'])
plt.ylabel('Model')
plt.title('Cross Validation Scores')

Le modèle GradientBoostingClassifier donne les meilleures performances en cross validation. 